In [1]:
import gpt_wrapper

gpt_wrapper.api_base = "http://mnlp-backend-938795011.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "3e510581-28fa-4342-9758-4fa131bc2f42"

In [2]:
from gpt_wrapper.chat import Chat
chat1 = Chat.create("Test Chat")

In [43]:
import pandas as pd
df = pd.read_csv('/home/rfaro/anaconda3/EPFL_ML/ml-project-2-betterthanpoli/data/qa_heatpumps_2.csv', sep = ";")
queries1 = df['Question'].tolist()
answers1 = df['Answer'].tolist()
df2 = pd.read_csv('/home/rfaro/anaconda3/EPFL_ML/ml-project-2-betterthanpoli/RL/RLHF/q&a_dataset.csv', sep = "; ")
queries2 = df2['Question'].tolist()
answers2 = df2['Answer'].tolist()

queries = queries1 + queries2
answers = answers1 + answers2

# data = list(zip(queries, answers))
data = []
for i in range(len(queries)):
    data.append([])
    data[i].append(queries[i])
    data[i].append(answers[i])

/tmp/ipykernel_1416445/2875029425.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv('/home/rfaro/anaconda3/EPFL_ML/ml-project-2-betterthanpoli/RL/RLHF/q&a_dataset.csv', sep = "; ")


In [53]:
data[788]

['"What are the challenges and solutions associated with the installation of heat pumps in various climates?',
 'The installation of heat pumps in various climates presents challenges related to efficiency and performance. In extremely cold conditions, heat pumps may experience reduced efficiency and heating capacity, frost formation on the outdoor evaporator, and potential compressor damage. These challenges can be mitigated by using supplemental heat sources, advanced defrosting technologies, and selecting heat pumps designed for low-temperature operation. In hot climates, ensuring adequate cooling capacity and managing high ambient temperatures are key considerations. Proper system design, sizing, and installation, along with the integration of thermal storage and renewable energy systems, can help address these challenges and ensure reliable performance across different climates."']

In [54]:
import tqdm
for i in tqdm.tqdm(range(787, len(data))):
    chat1 = Chat.create("Test Chat")
    prompt = data[i][0]
    good_answer = data[i][1]

    prompt += "Here is a good answer to this question: " + good_answer + " Generate a different but worse answer to the same prompt."
    instruction = "Make your answer not too informal but less clear, less complete, less engaging, less relevant."
    message = chat1.ask(prompt, instruction = instruction, model_args={
        "max_tokens": 60, # default is 100
         })
    data[i].append(str(message))

100%|██████████| 97/97 [02:35<00:00,  1.61s/it]


In [57]:
dataset = {'prompt': [], 'chosen': [], 'rejected': []}
for i in range(len(data)):
    dataset['prompt'].append(data[i][0])
    dataset['chosen'].append(data[i][1])
    dataset['rejected'].append(data[i][2])
    

In [58]:
import json
json.dump(dataset, open('dpo_dataset.json', 'w'))